In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier

In [14]:
df = pd.read_csv("glass_binary.csv")
df = df.rename({'Class': 'Y'}, axis=1) 
df["Y"].replace({'negative' : int(0), "positive" : int(1)}, inplace=True, regex=True)
df.head(10)

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Y
0,151588824,1287795.00,343036.0,140066.00000,73282.00,0.68931,804468.0,0.0,0.1224,0
1,15176423,129777.00,353812.0,121127.00000,73002.00,0.65205,852888.0,0.0,0.0000,0
2,152212996,1420795.00,382099.0,0.46976,71.77,0.11178,95726.0,0.0,0.0000,0
3,152222108,1321045.00,37716.0,0.79076,719884.00,0.13041,102452.0,0.0,0.0000,0
4,151755118,13.39,365935.0,11888.00000,727892.00,0.57132,827064.0,0.0,0.0561,0
5,152099096,1368925.00,3592.0,112139.00000,719604.00,0.08694,940044.0,0.0,0.0000,0
6,151755118,131506.00,360996.0,105077.00000,732372.00,0.57132,823836.0,0.0,0.0000,0
7,151910022,1390205.00,373119.0,117917.00000,721228.00,0.06210,889472.0,0.0,0.0000,0
8,151793844,1321045.00,347975.0,141029.00000,72638.00,0.58995,843204.0,0.0,0.0000,0
9,151898632,1358285.00,334954.0,123053.00000,72078.00,0.58995,890548.0,0.0,0.0000,0


In [15]:
print(df.shape)
y = df.values[:,-1].astype(int)
X = df.drop("Y", axis=1)

(214, 10)


In [16]:
from collections import Counter
counter = Counter(y)
for k,v in counter.items():
	per = v / len(y) * 100
	print('Class=%d, Count=%d, Percentage=%.3f%%' % (k, v, per))

Class=0, Count=185, Percentage=86.449%
Class=1, Count=29, Percentage=13.551%


In [17]:
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
model = DummyClassifier(strategy='most_frequent')
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

print('Mean Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Mean Accuracy: 0.865 (0.008)


In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
print(len(X_train))
print(len(X_test))

160
54


In [21]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [22]:
def evaluate_model(X_train, y_train, X_test, y_test, model):
	cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
	model.fit(X_train, y_train)
	return cross_val_score(model, X_test, y_test, scoring='accuracy', cv=cv, n_jobs=-1)

models, names = list(), list()
# Dummy
models.append(DummyClassifier(strategy='most_frequent'))
names.append('DUM')
# SVM
models.append(SVC(gamma='auto'))
names.append('SVM')
# KNN
models.append(KNeighborsClassifier())
names.append('KNN')
# Bagging
models.append(BaggingClassifier(n_estimators=100))
names.append('BAG')
# RF
models.append(RandomForestClassifier(n_estimators=100))
names.append(' RF')
# ET
models.append(ExtraTreesClassifier(n_estimators=100))
names.append(' ET')

results = list()
for i in range(len(models)):
	scores = evaluate_model(X_train, y_train, X_test, y_test, models[i])
	results.append(scores)
	print('>%s %.3f (%.3f)' % (names[i], scores.mean(), scores.std()))

>DUM 0.871 (0.043)
>SVM 0.877 (0.053)
>KNN 0.890 (0.049)
>BAG 0.982 (0.036)
> RF 0.982 (0.036)
> ET 0.994 (0.023)


## ToDo
#### Feature Selection
#### Clustering